In [28]:
import carla
import time

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

def spawn_vehicle(vehicle_index=0, spawn_index=0, x_offset=0, y_offset=0, pattern='vehicle.*model3*') -> carla.Vehicle:
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    spawn_point.location.x += x_offset
    spawn_point.location.y += y_offset
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

# Spawn target vehicle
target_vehicle = spawn_vehicle(x_offset=-50)

time.sleep(10)

# Spawn the vehicle
ego_vehicle = spawn_vehicle()

# Add the radar sensor
radar_bp = world.get_blueprint_library().find('sensor.other.radar')
radar_bp.set_attribute('horizontal_fov', '10')  # Horizontal field of view
radar_bp.set_attribute('vertical_fov', '10')    # Vertical field of view
radar_bp.set_attribute('range', '20')           # Maximum range

radar_transform = carla.Transform(carla.Location(x=2.0, z=1.0))
radar = world.spawn_actor(radar_bp, radar_transform, attach_to=ego_vehicle)

# Variable to store the minimum TTC
min_ttc = float('inf')

def radar_callback(data: carla.RadarMeasurement):
    global min_ttc, min_distance
    min_ttc = float('inf')

    for detection, i in zip(data, range(len(data))):
        absolute_speed = abs(detection.velocity)

        # Calculate TTC
        if absolute_speed != 0:
            ttc = detection.depth / absolute_speed
            print(ttc)
            if ttc < min_ttc:
                min_ttc = ttc

# Register the radar callback
radar.listen(radar_callback)
i=0
try:
    while True:
        # TTC threshold (e.g., 2 seconds)
        ttc_threshold = 1.0

        # Braking control
        if min_ttc < ttc_threshold:
            control = carla.VehicleControl()
            control.brake = 1.0  # Maximum braking
            ego_vehicle.apply_control(control)
            print("Emergency braking activated!")
            i += 1
            if i>10:
                break
        else:
            control = carla.VehicleControl()
            control.throttle = 0.5  # Maintain constant speed
            ego_vehicle.apply_control(control)
        
        time.sleep(0.05)
    time.sleep(2)
except KeyboardInterrupt:
    print("Keyboard interrupt detected.")
finally:
    radar.stop()
    radar.destroy()
    ego_vehicle.destroy()
    target_vehicle.destroy()


0.002824874527677178
249.55043699768385
119.12378913126544
127.19535531868092
168.3596552530046
146.86353419865986
153.85505665899146
97.9697046341888
186.6958746564054
103.07671306460269
222.41997224342478
279.4703676144772
151.59701231009757
146.17665430231855
274.0033573023395
243.4806874449066
230.6080996941368
109.77316391640962
378.9047927715487
278.9717610418909
338.7742457792082
240.4233590433361
492.6405516710439
595.4649164874012
846.8181185552137
1138.0774884538034
858.4533018940485
912.9049550759977
458.0103538083176
2215.5654692932876
2700.5803215269843
4987.442591521441
3280.540800845498
4457.303238727669
6369.870665665996
3389.0513661529108
16579.71786994763
21477.618359486223
23941.5991758825
10111.740654874737
3412.5487170603205
6883.7315450885835
4748.492280244853
5012.50127466668
4489.250656054393
3890.8463966900017
3207.153500057097
2255.385097364787
3894.028728035616
2086.822366028245
1544.8940422790695
1672.0139792326001
2472.229579713077
2056.2857932328743
2069.8